In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np
from torchvision.models.segmentation import deeplabv3_resnet50
from torchvision.models import ResNet50_Weights
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader
from dataset import get_loaders
from PIL import Image
from tqdm import tqdm  # 進度條

# === 損失函數：Focal + Dice ===
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.8, gamma=2):
        super().__init__()
        self.alpha, self.gamma = alpha, gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
    def forward(self, inputs, targets):
        b = self.bce(inputs, targets)
        pt = torch.exp(-b)
        return (self.alpha*(1-pt)**self.gamma * b).mean()

def dice_loss(pred, target, eps=1e-6):
    p = torch.sigmoid(pred)
    inter = (p*target).sum()
    union = p.sum() + target.sum()
    return 1 - (2*inter+eps)/(union+eps)

def mixed_loss(pred, target):
    return FocalLoss()(pred, target) + dice_loss(pred, target)

# === 評估指標 ===
def compute_iou(preds, masks, threshold=0.3):
    p = (torch.sigmoid(preds)>threshold).float()
    inter = (p*masks).sum(dim=(1,2,3))
    union = ((p+masks)>0).float().sum(dim=(1,2,3))
    return (inter/(union+1e-6)).mean().item()

def evaluate_prediction_error(preds, masks, px_per_cm=72):
    p = (torch.sigmoid(preds)>0.3).float()
    errs=[]
    for i in range(p.size(0)):
        pp = torch.nonzero(p[i,0], as_tuple=False)
        tt = torch.nonzero(masks[i,0], as_tuple=False)
        if len(pp)==0 or len(tt)==0: continue
        errs.append(abs(pp[:,0].float().mean()-tt[:,0].float().mean()).item())
    if not errs: return 0,0,0
    m = np.mean(errs)
    return m, np.mean(np.array(errs)<=0.5*px_per_cm)*100, np.mean(np.array(errs)<=1.0*px_per_cm)*100

# === 模型建立 ===
def get_model():
    return deeplabv3_resnet50(
        weights=None,
        weights_backbone=ResNet50_Weights.IMAGENET1K_V2,
        num_classes=1
    )

# === 主程式 ===
device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.makedirs("checkpoints/deeplabv3", exist_ok=True)
folds = ['Fold1','Fold2','Fold3','Fold4','Fold5']

for fold in folds:
    print(f"=== 開始訓練 {fold} ===")
    model = get_model().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # 先用原本的 get_loaders 拿 dataset
    train_ds, val_ds = get_loaders('processed_data', fold, batch_size=None)
    # 再自己包 DataLoader，加上 drop_last=True
    train_loader = DataLoader(train_ds.dataset, batch_size=4, shuffle=True, drop_last=True)
    val_loader   = DataLoader(val_ds.dataset,   batch_size=4, shuffle=False, drop_last=True)

    best_iou = 0.0
    patience = 30
    wait = 0

    for epoch in range(50):
        # ———— 訓練 ————
        model.train()
        total_loss = 0.0
        for imgs, masks in tqdm(train_loader, desc=f"{fold} Ep{epoch+1}", leave=False):
            imgs, masks = imgs.to(device), masks.to(device)
            out = model(imgs)['out']
            loss = mixed_loss(out, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # ———— 驗證 ————
        model.eval()
        sum_iou = sum_err = sum_a05 = sum_a10 = 0.0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                out = model(imgs)['out']
                sum_iou += compute_iou(out, masks)
                e,a05,a10 = evaluate_prediction_error(out, masks)
                sum_err += e
                sum_a05 += a05
                sum_a10 += a10

        n = len(val_loader)
        val_iou = sum_iou/n
        val_err = sum_err/n
        val_a05 = sum_a05/n
        val_a10 = sum_a10/n
        print(f"[{fold}][{epoch+1}/50] Loss={total_loss:.1f}  IOU={val_iou:.3f}  Err={val_err:.1f}px  <0.5cm={val_a05:.1f}%  <1cm={val_a10:.1f}%")

        # Early stopping
        if val_iou > best_iou:
            best_iou = val_iou
            wait = 0
            torch.save(model.state_dict(), f"checkpoints/deeplabv3/deeplabv3_{fold}_best.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"🛑 提前停止 {fold} 於 Epoch {epoch+1}")
                break

    # ———— 存預測圖 ————
    out_dir = f"predictions/deeplabv3/{fold}"
    os.makedirs(out_dir, exist_ok=True)
    model.eval()
    cnt = 1
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            preds = (torch.sigmoid(model(imgs)['out'])>0.3).float()
            for i in range(imgs.size(0)):
                orig = to_pil_image(imgs[i][0].cpu())
                gt   = to_pil_image(masks[i][0].cpu())
                pr   = to_pil_image(preds[i][0].cpu())
                w,h = orig.size
                combo = Image.new('L',(w*3,h))
                combo.paste(orig,(0,0)); combo.paste(gt,(w,0)); combo.paste(pr,(2*w,0))
                combo.save(f"{out_dir}/pred_{cnt}.png")
                cnt += 1

print("✅ 全部 Fold 訓練完成！")


=== 開始訓練 Fold1 ===


[Fold1][1/50] Loss=73.8  IOU=0.021  Err=72.4px  <0.5cm=11.4%  <1cm=38.6%


[Fold1][2/50] Loss=68.8  IOU=0.110  Err=10.8px  <0.5cm=100.0%  <1cm=100.0%


[Fold1][3/50] Loss=66.2  IOU=0.122  Err=12.9px  <0.5cm=97.7%  <1cm=100.0%


[Fold1][4/50] Loss=62.3  IOU=0.143  Err=10.4px  <0.5cm=97.7%  <1cm=100.0%


...

[Fold5][33/50] Loss=5.9  IOU=0.156  Err=8.9px  <0.5cm=100.0%  <1cm=100.0%


[Fold5][34/50] Loss=5.7  IOU=0.152  Err=8.9px  <0.5cm=100.0%  <1cm=100.0%


[Fold5][35/50] Loss=5.7  IOU=0.113  Err=10.6px  <0.5cm=100.0%  <1cm=100.0%


[Fold5][36/50] Loss=5.7  IOU=0.121  Err=10.3px  <0.5cm=97.9%  <1cm=100.0%


[Fold5][37/50] Loss=5.5  IOU=0.123  Err=10.6px  <0.5cm=100.0%  <1cm=100.0%
🛑 提前停止 Fold5 於 Epoch 37
✅ 全部 Fold 訓練完成！
